In [1]:
import re
import string
import numpy as np
from collections import Counter

In [14]:
class SpellChecker():
    
    def __init__(self, corpus_file):
        with open(corpus_file, 'r') as file:
            lines = file.readlines()
            words = []
            for line in lines:
                words += re.findall(r'\w+', line.lower()) #read as single words and input it in the words list
    
        self.vocabs = set(words) #get unique words
        self.word_count = Counter(words)
        N = float(sum(self.word_count.values())) #total words
        self.word_prob = {word: self.word_count[word] / N for word in self.vocabs}
    
    def first_edit(self, word):
        letters = string.ascii_lowercase                                  #lowercase alphabet
        split = [(word[:i], word[i:]) for i in range(len(word) + 1)]      #[('', 'bug'), ('b', 'ug'), ('bu', 'g'), ('bug', '')]
        
        #l is left (before comma) , r is right (after comma)
        delete = [l + r[1:] for l, r in split if r]                       #[ug, bg, bu]
        swap = [l + r[1] + r[0] + r[2:] for l, r in split if len(r) > 1]  #[ubg, bgu]
        replace = [l + c + r[1:] for l, r in split if r for c in letters] #[aug, bug, cug, ..., buz]
        insert = [l + c + r for l, r in split if r for c in letters]      #[abug, bbug, cbug, ..., bugz]
        
        return set(delete + swap + replace + insert) #unique output
    
    def second_edit(self,word):
        # run first edit two times
        return set(edit2 for edit1 in self.first_edit(word) for edit2 in self.first_edit(edit1))
    
    def check(self, word):
        correction = self.first_edit(word) or self.second_edit(word) or [word]
        
        #check if the correction is a valid word in the corpus
        valid_correction = [w for w in correction if w in self.vocabs]
        
        return sorted([(c, self.word_prob[c]) for c in valid_correction], 
                      key = lambda tup: tup[1],  #sort based on the word_prob
                      reverse = True) #biggest prob first
    
    def correct(self, word):
        return max(self.check(word), key = lambda tup: tup[1])

In [24]:
#input corpus
checker = SpellChecker('.txt')

In [21]:
checker.check('kata')

[('kala', 0.011363636363636364), ('kata', 0.011363636363636364)]

In [23]:
checker.correct('kat')

('kala', 0.011363636363636364)